<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/interpret_shap_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !pip install -r gdrive/MyDrive/NLP\ Stance\ Detection/xai_atanasova/requirements.txt

ERROR: Could not find a version that satisfies the requirement captum==0.2.0=0 (from versions: 0.0.1, 0.1.0, 0.2.0, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.5.0)
ERROR: No matching distribution found for captum==0.2.0=0


In [ ]:
%cd gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT

/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT


## install packages

In [2]:
!pip install captum==0.2.0
!pip install python-papi==5.5.1.2
!pip install lime==0.2.0.0

!pip install torch==1.4.0
!pip install transformers==3.5.0
!pip install wandb==0.9.4
!pip install scikit-learn==0.23.1
!pip install scipy==1.5.0
!pip install ekphrasis==0.5.1
!pip install emoji

     |████████████████████████████████| 1.4 MB 27.6 MB/s 
     |████████████████████████████████| 8.8 MB 27.1 MB/s 
  Created wheel for python-papi: filename=python_papi-5.5.1.2-cp37-cp37m-linux_x86_64.whl size=862005 sha256=9b18b72fc2fb4267b0410180859266b278819988fa9b19e6654413949c7fd8b3
  Stored in directory: /root/.cache/pip/wheels/02/16/ab/da4dc99a38cd930d78dd4c5e07db6c618a192c38faf1a37cfd
Successfully built python-papi
     |████████████████████████████████| 274 kB 25.9 MB/s 
     |████████████████████████████████| 2.0 MB 60.3 MB/s 
  Created wheel for lime: filename=lime-0.2.0.0-py3-none-any.whl size=284191 sha256=60204ee4c32148d023f397f1bf063560417881b7914f569e1046531ea7e790a5
  Stored in directory: /root/.cache/pip/wheels/05/1a/3f/6b78b5cf3a5b8ed95a487c2539755c9e97907e53594dfe8e35
Successfully built lime
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency 

     |████████████████████████████████| 753.4 MB 6.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3 MB 30.0 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 1.1 MB 60.3 MB/s 
     |████████████████████████████████| 2.9 MB 67.2 MB/s 
     |████████████████████████████████| 1.4 MB 35.2 MB/s 
     |███████████████████

## run LIME interpret on the saved pythorch model 

In [ ]:
import argparse
import json
import os
import random
from argparse import Namespace
from collections import defaultdict
from functools import partial

import numpy as np
import torch
from captum.attr import ShapleyValueSampling
from pypapi import events, papi_high as high
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import BertConfig, BertForSequenceClassification, \
    BertTokenizer, AutoModel,AutoTokenizer
from transformers.modeling_auto import AutoModelForSequenceClassification

In [ ]:
saliency_path = "/contentgdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT"
model_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.pt"

In [ ]:
checkpoint = torch.load(model_path,
                            map_location=lambda storage, loc: storage)

In [ ]:
args_model = {"seed": 4214, "target_merger": "CI_ESRX", 
 "cross_valid_num": 4, "batch_size": 16, "lr": 1e-05, "n_epochs": 2, "dropout": 0.5, "mlp_hidden": 16, "device": "cuda", 
 "bert_type": "vinai/bertweet-base", "dataset_name": "wtwt"}

In [ ]:
from argparse import Namespace


In [ ]:
model_args = Namespace(**args_model)

In [ ]:
!python /content/gdrive/MyDrive/NLP\ Stance\ Detection/xai_atanasova/saliency_gen/interpret_shap_mod.py 

## helper function

In [ ]:
class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model

    def forward(self, input):
        return self.model(input.long())


class BertModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(BertModelWrapper, self).__init__()
        self.model = model

    def forward(self, input):
        return self.model(input.long(), attention_mask=input > 0)[0]


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset_dir",
                        help="Path to the direcory with the datasets",
                        default='data/e-SNLI/dataset/',
                        type=str)
    parser.add_argument("--dataset", help="Which dataset", default='snli',
                        type=str, choices=['snli', 'imdb', 'tweet'])
    parser.add_argument("--split", help="Which split of the dataset",
                        default='test', type=str,
                        choices=['train', 'test'])
    parser.add_argument("--no_time",
                        help="Whether to output the time for generation in "
                             "flop",
                        action='store_true',
                        default=False)
    parser.add_argument("--model", help="Which model", default='transformer',
                        choices=['cnn', 'lstm', 'transformer'], type=str)
    parser.add_argument("--gpu", help="Flag for running on gpu",
                        action='store_true', default=False)
    parser.add_argument("--seed", help="Random seed", type=int, default=4214)
    parser.add_argument("--labels", help="Num of labels", type=int, default=4)
    parser.add_argument("--model_path",
                        help="Path to the model", default=None,
                        type=str)
    parser.add_argument("--output_dir",
                        help="Path where the saliencies will be serialized",
                        default='saliency_snli_lime',
                        type=str)
    parser.add_argument("--batch_size",
                        help="Batch size for explanation generation", type=int,
                        default=None)

    args = parser.parse_args()
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(args.seed)

    device = torch.device("cuda") if args.gpu else torch.device("cpu")

    #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
    print(args, flush=True)

    model_path = args.model_path
    model_name = model_path.split('/')[-1]

    model_output_shap = os.path.join(args.output_dir, 'trans_1_shap')
    print(model_output_shap)
    all_flops = generate_saliency(model_path, model_output_shap)

    print('FLOPS', np.average(all_flops), np.std(all_flops), flush=True)

usage: ipykernel_launcher.py [-h] [--dataset_dir DATASET_DIR]
                             [--dataset {snli,imdb,tweet}]
                             [--split {train,test}] [--no_time]
                             [--model {cnn,lstm,transformer}] [--gpu]
                             [--seed SEED] [--labels LABELS]
                             [--model_path MODEL_PATH]
                             [--output_dir OUTPUT_DIR]
                             [--batch_size BATCH_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-607a90a8-5e02-423e-b250-ee92f1d16978.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
def generate_saliency(model_path, saliency_path):

    # device = torch.device("cuda") if args.gpu else torch.device("cpu")
    device = torch.device("cuda")


    checkpoint = torch.load(model_path,
                            map_location=lambda storage, loc: storage)
    #model_args = Namespace(**checkpoint['args'])

    args_model = {"seed": 4214, "target_merger": "CI_ESRX", "labels":4,
     "cross_valid_num": 4, "batch_size": 16, "lr": 1e-05, "n_epochs": 2, "dropout": 0.5, "mlp_hidden": 16, "device": "cuda", 
     "bert_type": "vinai/bertweet-base", "dataset_name": "wtwt"}

    model_args = Namespace(**args_model)

    # model_args.batch_size = args.batch_size if args.batch_size != None else \
    #     model_args.batch_size

    # if args.model == 'transformer':
      
        # transformer_config = BertConfig.from_pretrained('vinai/bertweet-base',
        #                                                 num_labels=args.labels)
        # modelb = BertForSequenceClassification.from_pretrained(
        #     'vinai/bertweet-base', config=transformer_config).to(
        #     device)
        #modelb.load_state_dict(checkpoint['model'])
    transformer_config = BertConfig.from_pretrained('vinai/bertweet-base', num_labels=model_args.labels)
    modelb = AutoModelForSequenceClassification.from_pretrained('vinai/bertweet-base', config=transformer_config).to(device)
    
    embedding_size = modelb.bert.embeddings.word_embeddings.weight.size(1)
    print(modelb.bert.embeddings.word_embeddings.weight.size())
    new_embeddings = torch.FloatTensor(3, embedding_size).uniform_(-0.1, 0.1)
    print(new_embeddings)
    new_embedding_weight = torch.cat((modelb.bert.embeddings.word_embeddings.weight.data,new_embeddings), 0)
    modelb.bert.embeddings.word_embeddings.weight.data = new_embedding_weight
    print("Embedding Shape:", modelb.bert.embeddings.word_embeddings.weight.data.size())
    
    modelb.load_state_dict(checkpoint)

    model = BertModelWrapper(modelb)

    return model 



In [ ]:
ss = generate_saliency(model_path, saliency_path)

NameError: ignored

## starting everything anew

In [ ]:
!pip install transformers==3.5.0
!pip install wandb==0.9.4
!pip install scikit-learn==0.23.1
!pip install scipy==1.5.0
!pip install ekphrasis==0.5.1
!pip install emoji
import time

In [ ]:
import torch
import torch.nn as nn
import os, json, random, sys
import numpy as np
from transformers import AutoTokenizer, AdamW, AutoModel
from captum.attr import ShapleyValueSampling
from functools import partial
from bertloader import StanceDataset 
from collections import defaultdict
from tqdm import tqdm
from pypapi import events, papi_high as high

np.random.seed(4124)
random.seed(4124)
torch.manual_seed(4124)

device = torch.device(f"cuda")

usage: ipykernel_launcher.py [-h] [--seed SEED]
                             [--target_merger TARGET_MERGER]
                             [--test_mode TEST_MODE]
                             [--cross_valid_num CROSS_VALID_NUM]
                             [--batch_size BATCH_SIZE] [--lr LR]
                             [--n_epochs N_EPOCHS] [--dropout DROPOUT]
                             [--mlp_hidden MLP_HIDDEN] [--dummy_run]
                             [--device DEVICE] [--wandb] --bert_type BERT_TYPE
                             [--notarget] --dataset_name DATASET_NAME
ipykernel_launcher.py: error: the following arguments are required: --bert_type, --dataset_name


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
saliency_path = "/contentgdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT"
model_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.pt"Z

In [ ]:
!python /content/gdrive/MyDrive/NLP\ Stance\ Detection/Code_Kaushal/interpret_shap_siqi.py --target_merger=CI_ESRX --bert_type=vinai/bertweet-base --dataset_name=wtwt

Streaming output truncated to the last 5000 lines.
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0,

## iter

In [ ]:
!python /content/gdrive/MyDrive/NLP\ Stance\ Detection/Code_Kaushal/interpret_shap_siqi_mm.py --target_merger=CI_ESRX --bert_type=vinai/bertweet-base --dataset_name=wtwt

++++++++++++++++++++++
Target = True
++++++++++++++++++++++
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Loaded Bert Tokenizer
Embedding Shape: torch.Size([64004, 768])
{'comment': 0, 'unrelated': 1, 'support': 2, 'refute': 3} || {0: 'comment', 1: 'unrelated', 2: 'support', 3: 'refute'}
Length of train, valid, test: 27130 0 2226
Length of train, eval_set: 27130 2226
Before shuffling train[0] =  944165633889095681 | After shuffling train[0] =  612468363198099456
/content/gdrive/.shortcut-targets-by-id/1EfEUYvABxjeNO888ZJWSNM5fn97YqKQr/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target
1. Type of train dataset
Length of train: 27130
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Loaded Bert Tokenizer
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is depre

In [ ]:
!python /content/gdrive/MyDrive/NLP\ Stance\ Detection/Code_Kaushal/interpret_shap_siqi_mm.py --target_merger=CI_ESRX --bert_type=vinai/bertweet-base --dataset_name=wtwt

++++++++++++++++++++++
Target = True
++++++++++++++++++++++
******New script2*****
Downloading: 100% 558/558 [00:00<00:00, 858kB/s]
Downloading: 100% 843k/843k [00:00<00:00, 21.2MB/s]
Downloading: 100% 1.08M/1.08M [00:00<00:00, 26.0MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Loaded Bert Tokenizer
Downloading: 100% 543M/543M [00:12<00:00, 41.8MB/s]
Embedding Shape: torch.Size([64004, 768])
{'comment': 0, 'unrelated': 1, 'support': 2, 'refute': 3} || {0: 'comment', 1: 'unrelated', 2: 'support', 3: 'refute'}
Length of train, valid, test: 27130 0 2226
Length of train, eval_set: 27130 2226
Before shuffling train[0] =  944165633889095681 | After shuffling train[0] =  612468363198099456
/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target
1. Type of train dataset
Length of train: 27130
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-t